# NISQA Analysys

## Tổng kết thời gian

In [ ]:
from utils.tool import  get_episode_duration, get_mp3_duration_ffprobe
from concurrent.futures import ProcessPoolExecutor
from functools import partial
import os
filter_path = '/home4/tuanlha/EXpressiveTTS/dataRawProcess/04_denoise'
raw_path = '/home4/tuanlha/DataTest'
vad_path = '/home4/tuanlha/EXpressiveTTS/dataRawProcess/03_vad'

playlist_name = os.listdir(vad_path)
sorted(playlist_name)

In [ ]:
playlist_name = ['DuyThanhNguyenFinance']

### Tong thoi gian du lieu raw

In [ ]:
def process_episode(playlist, vad_path, episode_name):
    playlist_path = os.path.join(vad_path, playlist)
    episode_path = os.path.join(playlist_path, episode_name)
    duration = get_episode_duration(episode_path)
    print(f'Playlist: {playlist}, Episode: {episode_name}, Duration: {duration} seconds')
    return (playlist, episode_name, duration)

In [ ]:
dct_vad = {}
for playlist in playlist_name:
    playlist_path = os.path.join(vad_path, playlist)
    dct_vad[playlist] = {}
    with ProcessPoolExecutor() as executor:
        futures = []
        for episode_name in os.listdir(playlist_path):
            futures.append(executor.submit(process_episode, playlist, vad_path, episode_name))
        
        for future in futures:
            playlist, episode_name, duration = future.result()
            dct_vad[playlist][episode_name] = duration
    for episode_name in os.listdir(playlist_path):
        episode_path = os.path.join(playlist_path, episode_name)
        duration = get_episode_duration(episode_path)
        dct_vad[playlist][episode_name] = duration
        print(f'Playlist: {playlist}, Episode: {episode_name}, Duration: {duration} seconds')

In [ ]:
dct_raw = {}
for playlist in playlist_name:
    playlist_path = os.path.join(raw_path, playlist)
    dct_raw[playlist] = {}
    for episode_name in os.listdir(playlist_path):
        duration = get_mp3_duration_ffprobe(os.path.join(playlist_path, episode_name))
        episode_name = episode_name.rsplit('.', 1)[0]
        dct_raw[playlist][episode_name] = duration
        print(f'Playlist: {playlist}, Episode: {episode_name}, Duration: {duration} seconds')

In [ ]:
dct_filter = {}
for playlist in playlist_name:
    playlist_path = os.path.join(filter_path, playlist)
    dct_filter[playlist] = {}
    for episode_name in os.listdir(playlist_path):
        episode_path = os.path.join(playlist_path, episode_name)
        duration = get_episode_duration(episode_path)
        dct_filter[playlist][episode_name] = duration
        print(f'Playlist: {playlist}, Episode: {episode_name}, Duration: {duration} seconds')

### Save raw, vad, filter duration data

In [ ]:
### Compare durations
mean_filter_extract = {}
mean_vad_extract={}
for playlist in playlist_name:
    vad_percentage = []
    filter_percentage = []
    print(f'Playlist: {playlist}')
    for episode_name in dct_vad[playlist]:
        vad_duration = dct_vad[playlist][episode_name]
        raw_duration = dct_raw[playlist].get(episode_name, None)
        filter_duration = dct_filter[playlist].get(episode_name, None)
        vad_percentage.append(vad_duration * 100 / raw_duration)
        filter_percentage.append(filter_duration * 100 / raw_duration)
        if raw_duration is not None and filter_duration is not None:
            print(f'Episode: {episode_name[:10]}, Raw Duration: {raw_duration}, VAD Duration: {vad_duration*100/raw_duration}, Filter Duration: {filter_duration*100/raw_duration}')
        else:
            print(f'Episode: {episode_name[:10]} missing in raw or filter data')
    mean_filter_extract[playlist] = sum(filter_percentage) / len(filter_percentage) if filter_percentage else 0
    mean_vad_extract[playlist] = sum(vad_percentage) / len(vad_percentage) if vad_percentage else 0
print('Mean VAD Extract:', mean_vad_extract)
print('Mean Filter Extract:', mean_filter_extract)

In [ ]:
# import json 
# with open('dct_vad.json', 'w') as f:
#     json.dump(dct_vad, f, indent=4)
# with open('dct_filter.json', 'w') as f:
#     json.dump(dct_filter, f, indent=4)
# with open('dct_raw.json', 'w') as f:
#     json.dump(dct_raw, f, indent=4)

In [ ]:
# import json
# with open('dct_raw.json', 'r') as f:
#     dct_vad = json.load(f)

# #get all duration
# total_duration_vad = 0
# for playlist, episodes in dct_vad.items():
#     for episode_name, duration in episodes.items():
#         total_duration_vad += duration
# print(f'Total duration of VAD episodes: {total_duration_vad} seconds')  

## Thống kê NISQA

In [ ]:
import os
import pandas as pd
filter_path = '/home4/tuanlha/EXpressiveTTS/dataRawProcess/05_data_extract/HaveASipClean'

ls = []
for csv_file in os.listdir(filter_path):
    if csv_file.endswith('wvmos.csv'):
        csv_path = os.path.join(filter_path, csv_file)
        df = pd.read_csv(csv_path)
        df = df['score'].tolist()
        ls.append([csv_file, df])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

bins = [0.2*i for i in range (0, 25)]
def create_bins(data, file_name):
    # Đếm số phần tử trong mỗi khoảng
    counts, bin_edges = np.histogram(data, bins=bins)

    # Tạo nhãn cho các khoảng (ví dụ: 0.0-0.5, 0.5-1.0,...)
    bin_labels = [f"{bin_edges[i]:.1f}-{bin_edges[i+1]:.1f}" for i in range(len(bin_edges)-1)]

    # Vẽ biểu đồ
    plt.figure(figsize=(8, 5))
    bars = plt.bar(bin_labels, counts, color='skyblue', edgecolor='black')

    # Thêm số lượng lên từng cột
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                 f'{int(height)}', ha='center', va='bottom')

    plt.title('Phân phối giá trị theo khoảng', fontweight='bold')
    plt.xlabel(f'{file_name}')
    plt.ylabel('Số lượng phần tử')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

for csv_file, df in ls:
    create_bins(df, csv_file)

## Thống kê WV

In [ ]:
import os
import pandas as pd
filter_path = '/home4/tuanlha/EXpressiveTTS/dataRawProcess/05_data_extract/HaveASipClean'

ls = []
for csv_file in os.listdir(filter_path):
    if csv_file.endswith('mos.csv'):
        csv_path = os.path.join(filter_path, csv_file)
        df = pd.read_csv(csv_path)
        df = df['score'].tolist()
        ls.append([csv_file, df])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

bins = [0.2*i for i in range (10, 25)]
def create_bins(data, file_name):
    # Đếm số phần tử trong mỗi khoảng
    counts, bin_edges = np.histogram(data, bins=bins)

    # Tạo nhãn cho các khoảng (ví dụ: 0.0-0.5, 0.5-1.0,...)
    bin_labels = [f"{bin_edges[i]:.1f}-{bin_edges[i+1]:.1f}" for i in range(len(bin_edges)-1)]

    # Vẽ biểu đồ
    plt.figure(figsize=(8, 5))
    bars = plt.bar(bin_labels, counts, color='skyblue', edgecolor='black')

    # Thêm số lượng lên từng cột
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                 f'{int(height)}', ha='center', va='bottom')

    plt.title('Phân phối giá trị theo khoảng', fontweight='bold')
    plt.xlabel(f'{file_name}')
    plt.ylabel('Số lượng phần tử')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

for csv_file, df in ls:
    create_bins(df, csv_file)

In [ ]:
import os
import pandas as pd
filter_path = '/home4/tuanlha/EXpressiveTTS/dataRawProcess/05_data_extract/HaveASipClean'

ls = []
for csv_file in os.listdir(filter_path):
    if csv_file.endswith('results.csv'):
        csv_path = os.path.join(filter_path, csv_file)
        df = pd.read_csv(csv_path)
        df = df['mos_pred'].tolist()
        ls.append([csv_file, df])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

bins = [0.2*i for i in range (0, 25)]
def create_bins(data, file_name):
    # Đếm số phần tử trong mỗi khoảng
    counts, bin_edges = np.histogram(data, bins=bins)

    # Tạo nhãn cho các khoảng (ví dụ: 0.0-0.5, 0.5-1.0,...)
    bin_labels = [f"{bin_edges[i]:.1f}-{bin_edges[i+1]:.1f}" for i in range(len(bin_edges)-1)]

    # Vẽ biểu đồ
    plt.figure(figsize=(8, 5))
    bars = plt.bar(bin_labels, counts, color='skyblue', edgecolor='black')

    # Thêm số lượng lên từng cột
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                 f'{int(height)}', ha='center', va='bottom')

    plt.title('Phân phối giá trị theo khoảng', fontweight='bold')
    plt.xlabel(f'{file_name}')
    plt.ylabel('Số lượng phần tử')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

for csv_file, df in ls:
    create_bins(df, csv_file)

## play các audio có vấn đề

In [ ]:
import os
from IPython.display import Audio, display
import pandas as pd

In [ ]:
def run_audio(audio_path):
    if os.path.exists(audio_path):
        display(Audio(audio_path, autoplay=True))
    else:
        print(f"File {audio_path} does not exist.")

In [ ]:
filter_path = '/home4/tuanlha/EXpressiveTTS/dataRawProcess/05_data_extract/HaveASipClean'
audio_data = '/home4/tuanlha/EXpressiveTTS/dataRawProcess/04_vad/HaveASipClean'
cnt_episode=  0
for file_name in os.listdir(filter_path):
    if file_name.endswith('results.csv'):
        continue
    df = pd.read_csv(os.path.join(filter_path, file_name))
    print(df)
    df = df[df['score'] > 3.0]
    # df = df.tolist()

    episode_name = file_name.split('_')[0]
    episode_path = os.path.join(audio_data, episode_name)
    print(episode_name)
    cnt = 0
    for index, row in df.iterrows():
        file_path = os.path.join(episode_path, row['audio_path'])
        print(f'Playing {row["audio_path"]} audio score {row["score"]}:')
        display(Audio(file_path, autoplay=True))
        cnt+=1
        if cnt>4:
            break
    cnt_episode+=1
    if cnt_episode>4:
        break
    #https://youtu.be/DcCIb1cjKvg?si=7Jog8Mc6MM4ovNJH&t=1037
    #https://youtu.be/DcCIb1cjKvg?si=EwlihDEW_hxEKaQu&t=449

In [ ]:
filter_path = '/home4/tuanlha/EXpressiveTTS/dataRawProcess/05_data_extract/HaveASip'
audio_data = '/home4/tuanlha/EXpressiveTTS/dataRawProcess/05_transcript/HaveASip'
cnt_episode=  0
for file_name in os.listdir(filter_path):
    df = pd.read_csv(os.path.join(filter_path, file_name))
    df = df[df['mos_pred'] > 4.0]['deg']
    df = df.tolist()

    episode_name = file_name.split('_')[0]
    episode_path = os.path.join(audio_data, episode_name)
    print(episode_name)
    cnt = 0
    for item in df:
        file_path = os.path.join(episode_path, item)
        print(f'Playing {item} audio:')
        display(Audio(file_path, autoplay=True))
        cnt+=1
        if cnt>4:
            break
    cnt_episode+=1
    if cnt_episode>4:
        break
    #https://youtu.be/DcCIb1cjKvg?si=7Jog8Mc6MM4ovNJH&t=1037
    #https://youtu.be/DcCIb1cjKvg?si=EwlihDEW_hxEKaQu&t=449